In [3]:
def fetch_csv(service:str = "",ticket:str = "", params_plus:dict = {}, manual_login:tuple = None): # manual_login formát: (jméno, heslo)
    import requests
    from io import StringIO
    import os
    from dotenv import load_dotenv

    assert service != "", "Service is necessary"
    
    url = "https://ws.ujep.cz/ws/services/rest2" + service

    params = {
        "outputFormat":"CSV",
        "outputFormatEncoding":"utf-8"
    }

    params.update(params_plus)

    cookies = {}
    if ticket != "":
        cookies.update({"WSCOOKIE":ticket})

    load_dotenv()

    user = os.getenv("STAG_USER")
    password = os.getenv("STAG_PASSWORD")

    if user == None or password == None:
        auth = login_correction(manual_login)
    else:
        auth = (user, password)

    data = requests.get(url, params=params, cookies=cookies, auth=auth)

    wrap = StringIO(data.text)
    return wrap

# Manuální login override (technicky underride ale meh) pro testování
def login_correction(manual_login:tuple): #Testování, zda tuple má dva prvky a ty prvky zda jsou stringy

    if manual_login == None:
        return manual_login

    try:
        assert isinstance(manual_login[0], str)
        assert isinstance(manual_login[1], str)
    except:
        auth = None
    else:
        auth = (manual_login[0], manual_login[1])
    
    return auth


In [18]:
import polars as pl

ticket = "30088f13cc4a64c91aef019587bf2a31f7ff7055306e11abaef001d927dd099a"
auth = ("st101885", "x0301093100")

fakulta="PRF"

params_kateder = {
    "typPracoviste":"K",
    "zkratka":"%",
    "nadrazenePracoviste":fakulta
    }
print(params_kateder["nadrazenePracoviste"])
katedry_csv = pl.read_csv(fetch_csv(service="/ciselniky/getSeznamPracovist", params_plus=params_kateder, ticket=ticket, manual_login=auth), separator=";")
#print(katedry_csv.head())
katedry_list = katedry_csv.to_series(2)
katedry_list = katedry_list.to_list()
#print(katedry_list)


loner = katedry_list.pop(0)
params_rozvrh = {
    "stagUser": "F23112",
    "semestr":"%",
    "vsechnyCasyKonani":"false",
    "jenRozvrhoveAkce":"true",
    "vsechnyAkce":"false",
    "jenBudouciAkce":"false",
    "lang":"cs",
    "katedra":loner,
    "rok":"2023"
}
params_predmety = {
    "lang":"cs",
    "katedra":loner,
    "jenNabizeneECTSPrijezdy":"false",
    "rok":"2023"
}

excel_rozvrhy = pl.read_csv(fetch_csv(service="/rozvrhy/getRozvrhByKatedra", params_plus=params_rozvrh, ticket=ticket, manual_login=auth), separator=";")
excel_predmety = pl.read_csv(fetch_csv(service="/predmety/getPredmetyByKatedraFullInfo", params_plus=params_predmety), separator=";")
#excel_predmety.write_csv("source_testing/predmety-1.csv")
excel_predmety.head(5)

PRF


katedra,zkratka,rok,nazev,nazevDlouhy,vyukaZS,vyukaLS,kreditu,viceZapis,minObsazeni,garanti,garantiSPodily,garantiUcitIdno,prednasejici,prednasejiciSPodily,prednasejiciUcitIdno,cvicici,cviciciSPodily,cviciciUcitIdno,seminarici,seminariciSPodily,seminariciUcitIdno,schvalujiciUznani,schvalujiciUznaniUcitIdno,examinatori,examinatoriUcitIdno,podminujiciPredmety,vylucujiciPredmety,podminujePredmety,literatura,nahrazPredmety,metodyVyucovaci,metodyHodnotici,akreditovan,jednotekPrednasek,jednotkaPrednasky,jednotekCviceni,jednotkaCviceni,jednotekSeminare,jednotkaSeminare,anotace,typZkousky,maZapocetPredZk,formaZkousky,pozadavky,prehledLatky,predpoklady,ziskaneZpusobilosti,casovaNarocnost,predmetUrl,vyucovaciJazyky,poznamka,ectsZobrazit,ectsAkreditace,ectsNabizetUPrijezdu,poznamkaVerejna,skupinaAkreditace,skupinaAkreditaceKey,zarazenDoPrezencnihoStudia,zarazenDoKombinovanehoStudia,studijniOpory,praxePocetDnu,urovenNastavena,urovenVypoctena,automatickyUznavatZppZk,hodZaSemKombForma,semestr,pocetStudentu,aMax,bMax,cMax,aSkut,bSkut,cSkut
str,str,i64,str,str,str,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,str,i64,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,str,str,str,str,str,i64,str,str,str,i64,i64,i64
"""CNB""","""BK502""",2023,"""Fyziologie živ…","""Fyziologie živ…","""A""","""N""",6,"""NE""","""""","""'Ing. Stanisla…","""'Ing. Stanisla…","""8024""","""'Mgr. Marian R…","""'Mgr. Marian R…","""5275, 8024""","""'Mgr. Marian R…","""'Mgr. Marian R…","""5275, 8024""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""N""",12,"""HOD/SEM""",18,"""HOD/SEM""",0,"""HOD/SEM""","""Předmět poskyt…","""Zkouška""","""ANO""","""Kombinovaná""","""Písemný zápočt…","""1. Vnitřní pro…","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""N""","""A""","""""",0,"""""","""Bc.""","""A""","""""","""ZS""",0,"""""","""""","""""",0,0,0
"""CNB""","""BP2""",2023,"""Bakalářská prá…","""Bakalářská prá…","""A""","""N""",5,"""NE""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""KFY/BP2""","""""","""""","""N""",0,"""HOD/TYD""",0,"""HOD/TYD""",0,"""HOD/TYD""","""Cílem kurzu je…","""Zápočet""","""NE""","""Kombinovaná""","""Zápočet uděluj…","""""","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""A""","""N""","""""",0,"""""","""Bc.""","""A""","""""","""ZS""",4,"""""","""""","""""",4,0,0
"""CNB""","""BP502""",2023,"""Fyziologie živ…","""Fyziologie živ…","""A""","""N""",6,"""NE""","""""","""'Ing. Stanisla…","""'Ing. Stanisla…","""8024""","""'Mgr. Marian R…","""'Mgr. Marian R…","""5275, 8024""","""'Mgr. Marian R…","""'Mgr. Marian R…","""5275, 8024""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""N""",2,"""HOD/TYD""",3,"""HOD/TYD""",0,"""HOD/TYD""","""Předmět poskyt…","""Zkouška""","""ANO""","""Kombinovaná""","""Písemný zápočt…","""1. Vnitřní pro…","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""A""","""N""","""""",0,"""""","""Bc.""","""A""","""""","""ZS""",0,"""""","""""","""""",0,0,0
"""CNB""","""EXK""",2023,"""Exkurze""","""Exkurze""","""A""","""A""",1,"""NE""","""""","""'doc. Ing. Mar…","""'doc. Ing. Mar…","""2384""","""""","""""","""""","""""","""""","""""","""'doc. Ing. Mar…","""'doc. Ing. Mar…","""2384""","""""","""""","""""","""""","""""","""""","""""","""""","""KFY/EXK""","""""","""""","""N""",0,"""HOD/TYD""",1,"""HOD/TYD""",0,"""HOD/TYD""","""Cílem kurzu je…","""Zápočet""","""NE""","""Kombinovaná""","""""","""""","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""A""","""N""","""""",0,"""""","""Bc.""","""A""","""""","""ZS""",0,"""""","""""","""""",0,0,0
"""C

In [14]:
#for num,katedra in enumerate(katedry_list):
katedra = "CPPV"
params_rozvrh["katedra"] = katedra
params_predmety["katedra"] = katedra

temp_rozvrhy = pl.read_csv(fetch_csv(service="/rozvrhy/getRozvrhByKatedra", params_plus=params_rozvrh, ticket=ticket, manual_login=auth), separator=";")
temp_predmety = pl.read_csv(fetch_csv(service="/predmety/getPredmetyByKatedraFullInfo", params_plus=params_predmety), separator=";")
temp_predmety.head(3)

#     excel_predmety = excel_predmety.extend(temp_predmety)
#     excel_rozvrhy = excel_rozvrhy.extend(temp_rozvrhy)

# excel_rozvrhy.write_csv("source_tables/rozvrhy_fakulta.csv")
# excel_predmety.write_csv("source_tables/predmety_fakulta.csv")

katedra,zkratka,rok,nazev,nazevDlouhy,vyukaZS,vyukaLS,kreditu,viceZapis,minObsazeni,garanti,garantiSPodily,garantiUcitIdno,prednasejici,prednasejiciSPodily,prednasejiciUcitIdno,cvicici,cviciciSPodily,cviciciUcitIdno,seminarici,seminariciSPodily,seminariciUcitIdno,schvalujiciUznani,schvalujiciUznaniUcitIdno,examinatori,examinatoriUcitIdno,podminujiciPredmety,vylucujiciPredmety,podminujePredmety,literatura,nahrazPredmety,metodyVyucovaci,metodyHodnotici,akreditovan,jednotekPrednasek,jednotkaPrednasky,jednotekCviceni,jednotkaCviceni,jednotekSeminare,jednotkaSeminare,anotace,typZkousky,maZapocetPredZk,formaZkousky,pozadavky,prehledLatky,predpoklady,ziskaneZpusobilosti,casovaNarocnost,predmetUrl,vyucovaciJazyky,poznamka,ectsZobrazit,ectsAkreditace,ectsNabizetUPrijezdu,poznamkaVerejna,skupinaAkreditace,skupinaAkreditaceKey,zarazenDoPrezencnihoStudia,zarazenDoKombinovanehoStudia,studijniOpory,praxePocetDnu,urovenNastavena,urovenVypoctena,automatickyUznavatZppZk,hodZaSemKombForma,semestr,pocetStudentu,aMax,bMax,cMax,aSkut,bSkut,cSkut
str,str,i64,str,str,str,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,str,i64,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,str,str,str,i64,str,i64,str,str,str,i64,i64,i64
"""CPPV""","""APP""",2023,"""Asistentská pe…","""Asistentská pe…","""A""","""N""",2,"""NE""","""""","""'doc. PhDr. Ka…","""'doc. PhDr. Ka…","""8441""","""""","""""","""""","""""","""""","""""","""'RNDr. Eva Hej…","""'RNDr. Eva Hej…","""301, 425, 2776…","""""","""""","""""","""""","""""","""""","""""","""'ČAPEK, R. Ped…","""""","""""","""""","""N""",0,"""HOD/TYD""",0,"""HOD/TYD""",1,"""TYD/SEM""","""Obsahové zaměř…","""Zápočet""","""NE""","""Kombinovaná""","""řádné absolvov…","""Obsahové zaměř…","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""A""","""N""","""""",0,"""""","""Bc.""","""A""",0,"""ZS""",26,"""""","""""","""""",18,2,6
"""CPPV""","""DOUC""",2023,"""Doučování""","""Doučování""","""A""","""A""",2,"""NE""","""""","""""","""""","""""","""""","""""","""""","""'RNDr. Jiří Kr…","""'RNDr. Jiří Kr…","""303, 425, 2776…","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""A""",0,"""HOD/TYD""",2,"""HOD/TYD""",0,"""HOD/TYD""","""V rámci výběro…","""Zápočet""","""NE""","""Kombinovaná""","""""","""""","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""N""","""N""","""""",0,"""""","""""","""""",0,"""ZS""",0,"""""","""""","""""",0,0,0
"""CPPV""","""KAPP""",2023,"""Asistentská pe…","""Asistentská pe…","""A""","""N""",2,"""NE""","""""","""'doc. PhDr. Ka…","""'doc. PhDr. Ka…","""8441""","""""","""""","""""","""""","""""","""""","""'RNDr. Eva Hej…","""'RNDr. Eva Hej…","""301, 425, 2776…","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""N""",0,"""HOD/TYD""",0,"""HOD/TYD""",1,"""TYD/SEM""","""""","""Zápočet""","""NE""","""Kombinovaná""","""""","""""","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""N""","""A""","""""",0,"""""","""Bc.""","""""",0,"""ZS""",5,"""""","""""","""""",2,0,3


In [21]:
type(excel_rozvrhy.dtypes[0])

Int64

In [19]:
def type_check(dataframe1:"pl.DataFrame", dataframe2:"pl.DataFrame") -> list:
    types1 = dataframe1.dtypes
    types2 = dataframe2.dtypes
    problems = []
    for index in range(len(types1) - 1):
        if types1[index] != types2[index]:
            problems.append(dataframe1.columns[index])

    return problems


type_check(excel_rozvrhy, temp_rozvrhy)


ucitIdno
ucitIdno.ucitel
podilNaVyuce.ucitel


In [10]:
excel_predmety.vstack(temp_predmety)

katedra,zkratka,rok,nazev,nazevDlouhy,vyukaZS,vyukaLS,kreditu,viceZapis,minObsazeni,garanti,garantiSPodily,garantiUcitIdno,prednasejici,prednasejiciSPodily,prednasejiciUcitIdno,cvicici,cviciciSPodily,cviciciUcitIdno,seminarici,seminariciSPodily,seminariciUcitIdno,schvalujiciUznani,schvalujiciUznaniUcitIdno,examinatori,examinatoriUcitIdno,podminujiciPredmety,vylucujiciPredmety,podminujePredmety,literatura,nahrazPredmety,metodyVyucovaci,metodyHodnotici,akreditovan,jednotekPrednasek,jednotkaPrednasky,jednotekCviceni,jednotkaCviceni,jednotekSeminare,jednotkaSeminare,anotace,typZkousky,maZapocetPredZk,formaZkousky,pozadavky,prehledLatky,predpoklady,ziskaneZpusobilosti,casovaNarocnost,predmetUrl,vyucovaciJazyky,poznamka,ectsZobrazit,ectsAkreditace,ectsNabizetUPrijezdu,poznamkaVerejna,skupinaAkreditace,skupinaAkreditaceKey,zarazenDoPrezencnihoStudia,zarazenDoKombinovanehoStudia,studijniOpory,praxePocetDnu,urovenNastavena,urovenVypoctena,automatickyUznavatZppZk,hodZaSemKombForma,semestr,pocetStudentu,aMax,bMax,cMax,aSkut,bSkut,cSkut
str,str,i64,str,str,str,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,str,i64,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,str,str,str,str,str,i64,str,str,str,i64,i64,i64
"""CNB""","""BK502""",2023,"""Fyziologie živ…","""Fyziologie živ…","""A""","""N""",6,"""NE""","""""","""'Ing. Stanisla…","""'Ing. Stanisla…","""8024""","""'Mgr. Marian R…","""'Mgr. Marian R…","""5275, 8024""","""'Mgr. Marian R…","""'Mgr. Marian R…","""5275, 8024""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""N""",12,"""HOD/SEM""",18,"""HOD/SEM""",0,"""HOD/SEM""","""Předmět poskyt…","""Zkouška""","""ANO""","""Kombinovaná""","""Písemný zápočt…","""1. Vnitřní pro…","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""N""","""A""","""""",0,"""""","""Bc.""","""A""","""""","""ZS""",0,"""""","""""","""""",0,0,0
"""CNB""","""BP2""",2023,"""Bakalářská prá…","""Bakalářská prá…","""A""","""N""",5,"""NE""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""KFY/BP2""","""""","""""","""N""",0,"""HOD/TYD""",0,"""HOD/TYD""",0,"""HOD/TYD""","""Cílem kurzu je…","""Zápočet""","""NE""","""Kombinovaná""","""Zápočet uděluj…","""""","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""A""","""N""","""""",0,"""""","""Bc.""","""A""","""""","""ZS""",4,"""""","""""","""""",4,0,0
"""CNB""","""BP502""",2023,"""Fyziologie živ…","""Fyziologie živ…","""A""","""N""",6,"""NE""","""""","""'Ing. Stanisla…","""'Ing. Stanisla…","""8024""","""'Mgr. Marian R…","""'Mgr. Marian R…","""5275, 8024""","""'Mgr. Marian R…","""'Mgr. Marian R…","""5275, 8024""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""N""",2,"""HOD/TYD""",3,"""HOD/TYD""",0,"""HOD/TYD""","""Předmět poskyt…","""Zkouška""","""ANO""","""Kombinovaná""","""Písemný zápočt…","""1. Vnitřní pro…","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""A""","""N""","""""",0,"""""","""Bc.""","""A""","""""","""ZS""",0,"""""","""""","""""",0,0,0
"""CNB""","""EXK""",2023,"""Exkurze""","""Exkurze""","""A""","""A""",1,"""NE""","""""","""'doc. Ing. Mar…","""'doc. Ing. Mar…","""2384""","""""","""""","""""","""""","""""","""""","""'doc. Ing. Mar…","""'doc. Ing. Mar…","""2384""","""""","""""","""""","""""","""""","""""","""""","""""","""KFY/EXK""","""""","""""","""N""",0,"""HOD/TYD""",1,"""HOD/TYD""",0,"""HOD/TYD""","""Cílem kurzu je…","""Zápočet""","""NE""","""Kombinovaná""","""""","""""","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""A""","""N""","""""",0,"""""","""Bc.""","""A""","""""","""ZS""",0,"""""","""""","""""",0,0,0
"""C